In [56]:
import pandas as pd
import json

In [1]:
import os

openai_api_key = os.environ.get("OPENAI_API_KEY")

In [64]:
df = pd.read_csv('mfa13_export.csv')

In [5]:
df.head()

,Country,Category,Flow name,Flow code,Flow unit,1970,1971,1972,1973,1974,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,Coal,Exports,EXP,t,0,0,0,0,0,...,20000,46000,203000,231000,252000,425000,733000,1014000,1117000,772000
1,Afghanistan,Crop Residues,Exports,EXP,t,0,0,0,0,0,...,0,0,0,0,10,5,42,1,1,1
2,Afghanistan,Crops,Exports,EXP,t,141313,145933,194952,193574,223066,...,265841,172690,125194,226766,258412,286271,426735,496711,922134,1013622
3,Afghanistan,Excavated earthen materials (including soil) nec,Exports,EXP,t,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Afghanistan,Ferrous ores,Exports,EXP,t,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [11]:
agent = create_csv_agent(OpenAI(temperature=0, openai_api_key=openai_api_key), 
                         'mfa13_export.csv', 
                         verbose=True)

In [12]:
agent

AgentExecutor(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f8a6df4ca90>, verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f8a6df4ca90>, verbose=False, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df': '       Country                                          Category Flow name   \n0  Afghanistan                                              Coal   Exports  \\\n1  Afghanistan                                     Crop Residues   Exports   \n2  Afghanistan                                             Crops   Exports   \n3  Afghanistan  Excavated earthen materials (including soil) nec   Exports   \n4  Afghanistan                                      Ferrous ores   Exports   \n\n  Flow code Flow unit    1970    1971    1972    1973    1974  ...    2010   \n0       EXP         t       0    

In [13]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [14]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the rows
Action: python_repl_ast
Action Input: len(df)
Observation: 4994
Thought: I now know the final answer
Final Answer: There are 4994 rows in the dataframe.

> Finished chain.


'There are 4994 rows in the dataframe.'

In [15]:
agent.run("what is the latest year in the records?")



> Entering new AgentExecutor chain...
Thought: I need to find the last column in the dataframe
Action: python_repl_ast
Action Input: df.columns[-1]
Observation: 2019
Thought: I now know the final answer
Final Answer: 2019

> Finished chain.


'2019'

In [36]:
agent.run("which is the category with more export volume in 2019 in Afghanistan?")



> Entering new AgentExecutor chain...
Thought: I need to find the category with the highest export volume in 2019
Action: python_repl_ast
Action Input: df[df['Country'] == 'Afghanistan'].sort_values(by='2019', ascending=False).head(1)
Observation:        Country Category Flow name Flow code Flow unit    1970    1971    1972   
2  Afghanistan    Crops   Exports       EXP         t  141313  145933  194952  \

     1973    1974  ...    2010    2011    2012    2013    2014    2015   
2  193574  223066  ...  265841  172690  125194  226766  258412  286271  \

     2016    2017    2018     2019  
2  426735  496711  922134  1013622  

[1 rows x 55 columns]
Thought: I now know the final answer
Final Answer: The category with the highest export volume in 2019 in Afghanistan is Crops.

> Finished chain.


'The category with the highest export volume in 2019 in Afghanistan is Crops.'

In [40]:
agent.run("Which country exported more volume of petroleum in all years?")



> Entering new AgentExecutor chain...
Thought: I need to compare the volume of petroleum exported by each country
Action: python_repl_ast
Action Input: df[df['Flow name'] == 'Petroleum'].groupby('Country').sum()
Observation: Empty DataFrame
Columns: [Category, Flow name, Flow code, Flow unit, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
Index: []

[0 rows x 54 columns]
Thought: I need to find the country with the highest sum of all years
Action: python_repl_ast
Action Input: df[df['Flow name'] == 'Petroleum'].groupby('Country').sum().max()
Observation: Category     NaN
Flow name    NaN
Flow code    NaN
Flow unit    NaN
1970         NaN
1971         NaN
1972         NaN
1973         NaN
1974         NaN
1975         NaN
1976         N

'Saudi Arabia'

In [49]:
agent.run("What is the trend for category Petroleum for all years in Saudi Arabia?")



> Entering new AgentExecutor chain...
Thought: I need to find the row for Saudi Arabia and Petroleum
Action: python_repl_ast
Action Input: df[(df['Country'] == 'Saudi Arabia') & (df['Category'] == 'Petroleum')]
Observation:            Country   Category Flow name Flow code Flow unit       1970   
3820  Saudi Arabia  Petroleum   Exports       EXP         t  160037000  \

           1971       1972       1973       1974  ...       2010       2011   
3820  208986000  271582000  349334000  395627000  ...  290161000  352678000  \

           2012       2013       2014       2015       2016       2017   
3820  371346000  376616000  356884810  357324950  373200479  347463980  \

           2018       2019  
3820  367569820  350942840  

[1 rows x 55 columns]
Thought: I now need to look at the trend of the values
Action: python_repl_ast
Action Input: df[(df['Country'] == 'Saudi Arabia') & (df['Category'] == 'Petroleum')].T
Observation:                    3820
Country    Saudi Arabia
Category

'The trend for category Petroleum for all years in Saudi Arabia is increasing.'